In [1]:
import pandas as pd
import numpy as np

In [2]:
IMREP_df = pd.read_csv("../summary_data/50bp/IMREP/IMREP_TRB_merged_extracted_features.csv")
TRUST4_df = pd.read_csv("../summary_data/50bp/TRUST4/TRUST4_TRB_merged_extracted_features.csv")
MIXCR_df = pd.read_csv("../summary_data/50bp/MIXCR/MIXCR_TRB_merged_extracted_features.csv")
TCR_df = pd.read_csv("../summary_data/original/TCR_Seq/TCR_merged_extracted_features.csv")

In [3]:
# Select rows with nReads greater than 1
IMREP_df = IMREP_df[IMREP_df['nReads'] > 1]
TRUST4_df = TRUST4_df[TRUST4_df['nReads'] > 1]
MIXCR_df = MIXCR_df[MIXCR_df['nReads'] > 1]
TCR_df = TCR_df[TCR_df['nReads'] > 1]

In [4]:
# Rename nReads and frequencies columns according to tool for proper merging
IMREP_df = IMREP_df.rename(columns={"nReads": "nReads_IMREP"})
TRUST4_df = TRUST4_df.rename(columns={"nReads": "nReads_TRUST4"})
MIXCR_df = MIXCR_df.rename(columns={"nReads": "nReads_MIXCR"})
TCR_df = TCR_df.rename(columns={"nReads": "nReads_TCR"})

In [5]:
# Select rows on PBMC samples
TCR_df = TCR_df[TCR_df['Sample'].isin(['sample01','sample02','sample03','sample04','sample05'])]

Complete dataframe across all samples and tools

In [6]:
# Merge dataframes based on two key combination: Sample and CDR3. Outer join ensures no data is lost for instances that do not have overlap
merge_IMREP_TRUST4 = pd.merge(IMREP_df, TRUST4_df, how='outer', on=['Sample', 'CDR3']).fillna(0)
merge_IMREP_TRUST4_MIXCR = pd.merge(MIXCR_df, merge_IMREP_TRUST4, how='outer', on=['Sample', 'CDR3']).fillna(0)
merge_complete = pd.merge(TCR_df, merge_IMREP_TRUST4_MIXCR, how='outer', on=['Sample', 'CDR3']).fillna(0)

merge_complete

,Sample,CDR3,nReads_TCR,nReads_MIXCR,nReads_IMREP,nReads_TRUST4
0,sample01,CAWRGDTAQQPQHF,2.0,0.0,0.0,0.0
1,sample01,CASSRDSPETQYF,2.0,0.0,0.0,0.0
2,sample01,CASSYSGRALGTGELFF,2.0,0.0,0.0,0.0
3,sample01,CASSPDGGLRSPLHF,2.0,0.0,0.0,0.0
4,sample01,CASTPRGTVTSNQPQHF,4.0,0.0,0.0,6.0
...,...,...,...,...,...,...
49994,sample04,CASSLQGGQPQHF,0.0,0.0,0.0,2.0
49995,sample04,CASSLAMGGNQPQHF,0.0,0.0,0.0,2.0
49996,sample04,CASSSLGMGGYTF,0.0,0.0,0.0,2.0
49997,sample04,CASSYSPGPGDYGYTF,0.0,0.0,0.0,2.0


In [7]:
# Add the tissue type
merge_complete.loc[merge_complete['Sample']=='sample01','tissue'] = 'PBMC'
merge_complete.loc[merge_complete['Sample']=='sample02','tissue'] = 'PBMC'
merge_complete.loc[merge_complete['Sample']=='sample03','tissue'] = 'PBMC'
merge_complete.loc[merge_complete['Sample']=='sample04','tissue'] = 'PBMC'
merge_complete.loc[merge_complete['Sample']=='sample05','tissue'] = 'PBMC'

In [8]:
# Add T cell rich or poor tissue type
merge_complete.loc[merge_complete['Sample']=='sample01','tissue_type'] = 'T_cell_rich'
merge_complete.loc[merge_complete['Sample']=='sample02','tissue_type'] = 'T_cell_rich'
merge_complete.loc[merge_complete['Sample']=='sample03','tissue_type'] = 'T_cell_rich'
merge_complete.loc[merge_complete['Sample']=='sample04','tissue_type'] = 'T_cell_rich'
merge_complete.loc[merge_complete['Sample']=='sample05','tissue_type'] = 'T_cell_rich'

In [9]:
# Calculate total number of reads in each sample
total_reads = merge_complete[['Sample','nReads_TCR','nReads_MIXCR','nReads_IMREP','nReads_TRUST4']].groupby('Sample').sum().rename(columns={'nReads_TCR':'total_reads_TCR','nReads_MIXCR':'total_reads_MIXCR','nReads_IMREP':'total_reads_IMREP','nReads_TRUST4':'total_reads_TRUST4'})
total_reads

,total_reads_TCR,total_reads_MIXCR,total_reads_IMREP,total_reads_TRUST4
Sample,,,,
sample01,90577.0,62361.0,32598.0,18795.0
sample02,87762.0,37032.0,9813.0,4778.0
sample03,305953.0,42942.0,11118.0,20085.0
sample04,104779.0,103.0,234.0,84.0
sample05,18617.0,15.0,38.0,4.0


In [10]:
# Merge dataframes 
merge = pd.merge(merge_complete, total_reads, how='outer', on=['Sample']).fillna(0)

# Calculate frequency of CDR3 reads with respect to CDR3s that occur more than once
merge['frequency_TCR'] = merge['nReads_TCR'] / (merge['total_reads_TCR'] * 1.0)
merge['frequency_MIXCR'] = merge['nReads_MIXCR'] / (merge['total_reads_MIXCR'] * 1.0)
merge['frequency_IMREP'] = merge['nReads_IMREP'] / (merge['total_reads_IMREP'] * 1.0)
merge['frequency_TRUST4'] = merge['nReads_TRUST4'] / (merge['total_reads_TRUST4'] * 1.0)  
merge.fillna(0, inplace=True)
merge

,Sample,CDR3,nReads_TCR,nReads_MIXCR,nReads_IMREP,nReads_TRUST4,tissue,tissue_type,total_reads_TCR,total_reads_MIXCR,total_reads_IMREP,total_reads_TRUST4,frequency_TCR,frequency_MIXCR,frequency_IMREP,frequency_TRUST4
0,sample01,CAWRGDTAQQPQHF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,62361.0,32598.0,18795.0,0.000022,0.0,0.000000,0.000000
1,sample01,CASSRDSPETQYF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,62361.0,32598.0,18795.0,0.000022,0.0,0.000000,0.000000
2,sample01,CASSYSGRALGTGELFF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,62361.0,32598.0,18795.0,0.000022,0.0,0.000000,0.000000
3,sample01,CASSPDGGLRSPLHF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,62361.0,32598.0,18795.0,0.000022,0.0,0.000000,0.000000
4,sample01,CASTPRGTVTSNQPQHF,4.0,0.0,0.0,6.0,PBMC,T_cell_rich,90577.0,62361.0,32598.0,18795.0,0.000044,0.0,0.000000,0.000319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,sample05,CASSDGTGREQYF,0.0,0.0,4.0,0.0,PBMC,T_cell_rich,18617.0,15.0,38.0,4.0,0.000000,0.0,0.105263,0.000000
49995,sample05,CASSGEIQETQYF,0.0,0.0,2.0,0.0,PBMC,T_cell_rich,18617.0,15.0,38.0,4.0,0.000000,0.0,0.052632,0.000000
49996,sample05,CSF,0.0,0.0,8.0,0.0,PBMC,T_cell_rich,18617.0,15.0,38.0,4.0,0.000000,0.0,0.210526,0.000000
49997,sample05,CASSLHGGAGEQYF,0.0,0.0,2.0,0.0,PBMC,T_cell_rich,18617.0,15.0,38.0,4.0,0.000000,0.0,0.052632,0.000000


Differentiate monoclonal and polyclonal repertoire

In [11]:
# Calculate Shannon-Wiener index
diversity_TCR = merge[['Sample','tissue','tissue_type','CDR3','nReads_TCR','total_reads_TCR','frequency_TCR']]
diversity_TCR = diversity_TCR[diversity_TCR.nReads_TCR != 0]
clonotype_count_TCR = diversity_TCR.groupby(['Sample'], sort=False).size().reset_index(name='clonotype_count_TCR')

diversity_TCR['shannon_index_TCR'] = -(diversity_TCR['frequency_TCR']*np.log(diversity_TCR['frequency_TCR']))
shannon_TCR = diversity_TCR.groupby(['Sample']).agg({'shannon_index_TCR':'sum'}).reset_index().rename(columns={'':"shannon_index_TCR"})
shannon_TCR = pd.merge(shannon_TCR, clonotype_count_TCR, on=['Sample'])

# Define monoclonal sample as the shannon_index < 2, polyclonal sample as the shannon_index >= 2
shannon_TCR['repertoire_type'] = ['monoclonal' if x < 2 else 'polyclonal' for x  in shannon_TCR['shannon_index_TCR']]
repertoire_type = shannon_TCR[['Sample','repertoire_type']]
shannon_TCR

,Sample,shannon_index_TCR,clonotype_count_TCR,repertoire_type
0,sample01,1.072675,3001,monoclonal
1,sample02,1.001202,3229,monoclonal
2,sample03,0.960516,7697,monoclonal
3,sample04,9.598323,26802,polyclonal
4,sample05,8.005115,5440,polyclonal


In [12]:
# Generate metadata 
metadata = pd.merge(merge, repertoire_type, how='outer', on=['Sample'])
metadata.loc[:,'class'] = metadata["tissue_type"] +"_"+ metadata["repertoire_type"]

metadata.to_csv('../summary_data/50bp/all_tools_TRB.csv', index=False)
metadata

,Sample,CDR3,nReads_TCR,nReads_MIXCR,nReads_IMREP,nReads_TRUST4,tissue,tissue_type,total_reads_TCR,total_reads_MIXCR,total_reads_IMREP,total_reads_TRUST4,frequency_TCR,frequency_MIXCR,frequency_IMREP,frequency_TRUST4,repertoire_type,class
0,sample01,CAWRGDTAQQPQHF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,62361.0,32598.0,18795.0,0.000022,0.0,0.000000,0.000000,monoclonal,T_cell_rich_monoclonal
1,sample01,CASSRDSPETQYF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,62361.0,32598.0,18795.0,0.000022,0.0,0.000000,0.000000,monoclonal,T_cell_rich_monoclonal
2,sample01,CASSYSGRALGTGELFF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,62361.0,32598.0,18795.0,0.000022,0.0,0.000000,0.000000,monoclonal,T_cell_rich_monoclonal
3,sample01,CASSPDGGLRSPLHF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,62361.0,32598.0,18795.0,0.000022,0.0,0.000000,0.000000,monoclonal,T_cell_rich_monoclonal
4,sample01,CASTPRGTVTSNQPQHF,4.0,0.0,0.0,6.0,PBMC,T_cell_rich,90577.0,62361.0,32598.0,18795.0,0.000044,0.0,0.000000,0.000319,monoclonal,T_cell_rich_monoclonal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,sample05,CASSDGTGREQYF,0.0,0.0,4.0,0.0,PBMC,T_cell_rich,18617.0,15.0,38.0,4.0,0.000000,0.0,0.105263,0.000000,polyclonal,T_cell_rich_polyclonal
49995,sample05,CASSGEIQETQYF,0.0,0.0,2.0,0.0,PBMC,T_cell_rich,18617.0,15.0,38.0,4.0,0.000000,0.0,0.052632,0.000000,polyclonal,T_cell_rich_polyclonal
49996,sample05,CSF,0.0,0.0,8.0,0.0,PBMC,T_cell_rich,18617.0,15.0,38.0,4.0,0.000000,0.0,0.210526,0.000000,polyclonal,T_cell_rich_polyclonal
49997,sample05,CASSLHGGAGEQYF,0.0,0.0,2.0,0.0,PBMC,T_cell_rich,18617.0,15.0,38.0,4.0,0.000000,0.0,0.052632,0.000000,polyclonal,T_cell_rich_polyclonal


Calculate Shannon-Wiener index and absolute error for each sample across different tools

In [13]:
# MIXCR
diversity_MIXCR = merge[['Sample','tissue','tissue_type','CDR3','nReads_MIXCR','total_reads_MIXCR','frequency_MIXCR']]
diversity_MIXCR = diversity_MIXCR[diversity_MIXCR.nReads_MIXCR != 0]
clonotype_count_MIXCR = diversity_MIXCR.groupby(['Sample'], sort=False).size().reset_index(name='clonotype_count_tool')

diversity_MIXCR['shannon_index_tool'] = -(diversity_MIXCR['frequency_MIXCR']*np.log(diversity_MIXCR['frequency_MIXCR']))
shannon_MIXCR = diversity_MIXCR.groupby(['Sample']).agg({'shannon_index_tool':'sum'}).reset_index().rename(columns={'':"shannon_index_tool"})
shannon_MIXCR = pd.merge(shannon_MIXCR, clonotype_count_MIXCR, on=['Sample'])
shannon_MIXCR['tool'] = 'MIXCR'

# IMREP
diversity_IMREP = merge[['Sample','tissue','tissue_type','CDR3','nReads_IMREP','total_reads_IMREP','frequency_IMREP']]
diversity_IMREP = diversity_IMREP[diversity_IMREP.nReads_IMREP != 0]
clonotype_count_IMREP = diversity_IMREP.groupby(['Sample'], sort=False).size().reset_index(name='clonotype_count_tool')

diversity_IMREP['shannon_index_tool'] = -(diversity_IMREP['frequency_IMREP']*np.log(diversity_IMREP['frequency_IMREP']))
shannon_IMREP = diversity_IMREP.groupby(['Sample']).agg({'shannon_index_tool':'sum'}).reset_index().rename(columns={'':"shannon_index_tool"})
shannon_IMREP = pd.merge(shannon_IMREP, clonotype_count_IMREP, on=['Sample'])
shannon_IMREP['tool'] = 'IMREP'

# TRUST4
diversity_TRUST4 = merge[['Sample','tissue','tissue_type','CDR3','nReads_TRUST4','total_reads_TRUST4','frequency_TRUST4']]
diversity_TRUST4 = diversity_TRUST4[diversity_TRUST4.nReads_TRUST4 != 0]
clonotype_count_TRUST4 = diversity_TRUST4.groupby(['Sample'], sort=False).size().reset_index(name='clonotype_count_tool')

diversity_TRUST4['shannon_index_tool'] = -(diversity_TRUST4['frequency_TRUST4']*np.log(diversity_TRUST4['frequency_TRUST4']))
shannon_TRUST4 = diversity_TRUST4.groupby(['Sample']).agg({'shannon_index_tool':'sum'}).reset_index().rename(columns={'':"shannon_index_tool"})
shannon_TRUST4 = pd.merge(shannon_TRUST4, clonotype_count_TRUST4, on=['Sample'])
shannon_TRUST4['tool'] = 'TRUST4'

diversity = pd.concat([shannon_MIXCR,shannon_IMREP,shannon_TRUST4])
diversity = pd.merge(diversity, shannon_TCR, how='outer', on=['Sample'])
diversity = diversity.fillna(0)
tissue_type = merge[['Sample','tissue','tissue_type']].drop_duplicates(keep='first')
diversity = pd.merge(diversity, tissue_type, how='inner', on=['Sample'])
diversity['absolute_error'] = np.abs(diversity['shannon_index_TCR'] - diversity['shannon_index_tool'])
diversity.loc[:,'class'] = diversity["tissue_type"] +"_"+ diversity["repertoire_type"]
diversity.to_csv('../summary_data/50bp/all_tools_TRB_diversity.csv', index=False)

diversity

,Sample,shannon_index_tool,clonotype_count_tool,tool,shannon_index_TCR,clonotype_count_TCR,repertoire_type,tissue,tissue_type,absolute_error,class
0,sample01,0.633183,871,MIXCR,1.072675,3001,monoclonal,PBMC,T_cell_rich,0.439491,T_cell_rich_monoclonal
1,sample01,0.819503,576,IMREP,1.072675,3001,monoclonal,PBMC,T_cell_rich,0.253172,T_cell_rich_monoclonal
2,sample01,1.038436,411,TRUST4,1.072675,3001,monoclonal,PBMC,T_cell_rich,0.034239,T_cell_rich_monoclonal
3,sample02,0.438262,621,MIXCR,1.001202,3229,monoclonal,PBMC,T_cell_rich,0.562940,T_cell_rich_monoclonal
4,sample02,2.110875,910,IMREP,1.001202,3229,monoclonal,PBMC,T_cell_rich,1.109674,T_cell_rich_monoclonal
5,sample02,2.118826,392,TRUST4,1.001202,3229,monoclonal,PBMC,T_cell_rich,1.117625,T_cell_rich_monoclonal
6,sample03,0.332456,389,MIXCR,0.960516,7697,monoclonal,PBMC,T_cell_rich,0.628061,T_cell_rich_monoclonal
7,sample03,1.540740,733,IMREP,0.960516,7697,monoclonal,PBMC,T_cell_rich,0.580224,T_cell_rich_monoclonal
8,sample03,0.494763,309,TRUST4,0.960516,7697,monoclonal,PBMC,T_cell_rich,0.465754,T_cell_rich_monoclonal
9,sample04,3.483993,40,MIXCR,9.598323,26802,polyclonal,PBMC,T_cell_rich,6.114330,T_cell_rich_polyclonal
